In [ ]:
!pip install -q espnet datasets soundfile scikit-learn editdistance

In [ ]:
!git clone https://github.com/espnet/espnet.git
%cd espnet
!pip install -e .

fatal: destination path 'espnet' already exists and is not an empty directory.
/content/espnet
Obtaining file:///content/espnet
  Preparing metadata (setup.py) ... done
  Attempting uninstall: espnet
    Found existing installation: espnet 202409
    Uninstalling espnet-202409:
      Successfully uninstalled espnet-202409
  Running setup.py develop for espnet


In [ ]:
import os
import torch
import librosa
import soundfile as sf
from pathlib import Path
from sklearn.model_selection import train_test_split
import yaml
import sentencepiece as spm
import editdistance

In [ ]:
def  download_marathi_dataset():
    url = "https://www.cse.iitb.ac.in/~pjyothi/indiccorpora/Marathi.tgz"
    tgz_path = "Marathi.tgz"
    os.makedirs("marathi_data", exist_ok=True)
    !wget -O {tgz_path} {url}
    !tar -xvf Marathi.tgz -C marathi_data


download_marathi_dataset()

--2024-11-30 07:45:18--  https://www.cse.iitb.ac.in/~pjyothi/indiccorpora/Marathi.tgz
Resolving www.cse.iitb.ac.in (www.cse.iitb.ac.in)... 103.21.127.134
Connecting to www.cse.iitb.ac.in (www.cse.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 623353344 (594M) [application/x-gzip]
Saving to: ‘Marathi.tgz’

Marathi.tgz         100%[===================>] 594.48M  11.2MB/s    in 72s     

2024-11-30 07:46:32 (8.23 MB/s) - ‘Marathi.tgz’ saved [623353344/623353344]

Marathi/
Marathi/._College-Students.tgz
Marathi/College-Students.tgz
Marathi/._Rural-Low-Income-Workers.tgz
Marathi/Rural-Low-Income-Workers.tgz
Marathi/._Urban-Low-Income-Workers.tgz
Marathi/Urban-Low-Income-Workers.tgz


In [ ]:
RESAMPLED_AUDIO_DIR = "resampled_audio"
os.makedirs(RESAMPLED_AUDIO_DIR, exist_ok=True)

In [ ]:
def resample_audio(input_path, output_path, target_sr=16000):
    audio, sr = librosa.load(input_path, sr=None)
    if sr != target_sr:
        audio = librosa.resample(y=audio, orig_sr=sr, target_sr=target_sr)
    sf.write(output_path, audio, target_sr)
    return output_path

In [ ]:
#processing college students' dataset from the tar file
def process_tar_dataset():
    student_data = {
        'path': [],
        'sentence': []
    }

    temp_dir = "temp_college"
    os.makedirs(temp_dir, exist_ok=True)

    !tar -xvf marathi_data/Marathi/College-Students.tgz -C {temp_dir}

    for audio_file in Path(temp_dir).glob('**/*.3gp'):
        transcript_file = audio_file.with_suffix('.txt')

        if transcript_file.exists():
            new_filename = f"college_{audio_file.stem}.wav"
            resampled_path = os.path.join(RESAMPLED_AUDIO_DIR, new_filename)

            if not os.path.exists(resampled_path):
                resample_audio(str(audio_file), resampled_path)

            with open(transcript_file, 'r', encoding='utf-8') as f:
                transcript = f.read().strip()

            student_data['path'].append(resampled_path)
            student_data['sentence'].append(transcript)

    !rm -rf {temp_dir}

    return student_data

student_data = process_tar_dataset()

Streaming output truncated to the last 5000 lines.
College-Students/2740_02_1.3gp
College-Students/2740_02_1.txt
College-Students/2740_03_1.3gp
College-Students/2740_03_1.txt
College-Students/2740_04_1.3gp
College-Students/2740_04_1.txt
College-Students/2740_05_1.3gp
College-Students/2740_05_1.txt
College-Students/2740_06_1.3gp
College-Students/2740_06_1.txt
College-Students/2740_07_1.3gp
College-Students/2740_07_1.txt
College-Students/2740_08_1.3gp
College-Students/2740_08_1.txt
College-Students/2740_09_1.3gp
College-Students/2740_09_1.txt
College-Students/2740_10_1.3gp
College-Students/2740_10_1.txt
College-Students/2741_01_1.3gp
College-Students/2741_01_1.txt
College-Students/2741_02_1.3gp
College-Students/2741_02_1.txt
College-Students/2741_03_1.3gp
College-Students/2741_03_1.txt
College-Students/2741_04_1.3gp
College-Students/2741_04_1.txt
College-Students/2741_05_1.3gp
College-Students/2741_05_1.txt
College-Students/2741_06_1.3gp
College-Students/2741_06_1.txt
College-Students/27

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/

In [ ]:
#splitting data into training-testing-validation splits with seed = 64
train_paths, temp_paths, train_sentences, temp_sentences = train_test_split(
    student_data['path'], student_data['sentence'],
    test_size=0.2, random_state=64
)

valid_paths, test_paths, valid_sentences, test_sentences = train_test_split(
    temp_paths, temp_sentences,
    test_size=0.5, random_state=64
)

print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(valid_paths)}")
print(f"Test samples: {len(test_paths)}")

Train samples: 23648
Validation samples: 2956
Test samples: 2957


In [ ]:
#preparing dataset in ESPnet format
def prepare_dataset(paths, sentences, split_name):
    os.makedirs(f"data/{split_name}", exist_ok=True)

    with open(f"data/{split_name}/wav.scp", "w") as wav_file, \
         open(f"data/{split_name}/text", "w") as text_file, \
         open(f"data/{split_name}/utt2spk", "w") as utt2spk_file:
        for idx, (path, sentence) in enumerate(zip(paths, sentences)):
            uttid = f"utterance_{idx}"
            wav_file.write(f"{uttid} {path}\n")
            text_file.write(f"{uttid} {sentence}\n")
            utt2spk_file.write(f"{uttid} global\n")

    ##creating shape file with audio durations
    with open(f"data/{split_name}/speech.scp.shape", "w") as f:
        for idx, path in enumerate(paths):
            duration = librosa.get_duration(filename=path)
            shape = int(duration * 16000)
            f.write(f"utterance_{idx}\t{shape}\n")

prepare_dataset(train_paths, train_sentences, "train")
prepare_dataset(valid_paths, valid_sentences, "valid")
prepare_dataset(test_paths, test_sentences, "test")

In [ ]:
#training SentencePiece tokenikzer
os.makedirs("data/train", exist_ok=True)
with open("data/train/text_for_tokens.txt", "w", encoding="utf-8") as f:
    for text in train_sentences:
        f.write(f"{text}\n")

spm.SentencePieceTrainer.train(
    input="data/train/text_for_tokens.txt",
    model_prefix="data/train/marathi_tokenizer",
    vocab_size=1000,
    character_coverage=1.0,
    model_type='unigram'
)

#creating list of tokens
with open("data/train/tokens.txt", "w", encoding="utf-8") as f:
    with open("data/train/marathi_tokenizer.vocab", "r", encoding="utf-8") as vocab:
        for line in vocab:
            token = line.split('\t')[0]
            f.write(f"{token}\n")

In [ ]:
config = {
    "optim": "adam",
    "optim_conf": {
        "lr": 5e-5,
        "weight_decay": 0.000001
    },
    "max_epoch": 40,
    "batch_size": 32,
    "num_workers": 8,
    "accum_grad": 2,    #gradient accumulation over 2 steps
    "use_amp": True,    #enabling automatic mixed precision training to reduce memory usage
    "frontend": "default",
    "specaug_conf": {},
    "encoder": "conformer",
    "decoder": "rnn",
    "token_type": "char",
    "preprocessor": "default",
    "use_tensorboard": True,
    "patience": 5,    #waits 5 epochs before making changes
    "val_scheduler_criterion": ["valid", "loss", "min"],    #adjusting learning rate based on validation loss
    "early_stopping_criterion": ["valid", "loss", "min", 3],    #stopping if validation loss does not improve
    "best_model_criterion": [["valid", "loss", "min"]]    #saving model with lowest validation loss


}

In [ ]:
#saving configuration
os.makedirs("conf", exist_ok=True)
with open("conf/train_asr.yaml", "w") as f:
    yaml.dump(config, f)

In [ ]:
!python -m espnet2.bin.asr_train \
    --config conf/train_asr.yaml \
    --train_data_path_and_name_and_type data/train/wav.scp,speech,sound \
    --train_data_path_and_name_and_type data/train/text,text,text \
    --valid_data_path_and_name_and_type data/valid/wav.scp,speech,sound \
    --valid_data_path_and_name_and_type data/valid/text,text,text \
    --token_list data/train/tokens.txt \
    --train_shape_file data/train/speech.scp.shape \
    --valid_shape_file data/valid/speech.scp.shape \
    --fold_length 160000 \
    --output_dir exp/marathi_asr \
    --ngpu 1

Streaming output truncated to the last 5000 lines.
[d620ee9137ba] 2024-11-30 10:56:39,488 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
[d620ee9137ba] 2024-11-30 10:56:39,489 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
[d620ee9137ba] 2024-11-30 10:56:39,738 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
[d620ee9137ba] 2024-11-30 10:56:39,739 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to

#Testing

In [ ]:
!python -m espnet2.bin.asr_inference \
    --asr_model_file exp/marathi_asr/valid.loss.best.pth \
    --data_path_and_name_and_type data/test/wav.scp,speech,sound \
    --output_dir exp/marathi_asr/decode_test \
    --allow_variable_data_keys true

Streaming output truncated to the last 5000 lines.
2024-11-30 13:20:07,807 (beam_search:483) INFO: best hypo: कोलगेट-पामोलिव्हइंडियालिमिटेड-आर्बिट्रेजमाहितीदे.

2024-11-30 13:20:07,812 (asr_inference:509) INFO: speech length: 36480
2024-11-30 13:20:07,840 (beam_search:428) INFO: decoder input length: 70
2024-11-30 13:20:07,840 (beam_search:429) INFO: max output length: 70
2024-11-30 13:20:07,840 (beam_search:430) INFO: min output length: 0
2024-11-30 13:20:07,841 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
2024-11-30 13:20:07,841 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
2024-11-30 13:20:09,085 (beam_search:444) INFO: end detected at 22
2024-11-30 13:20:09,086 (beam_searc

In [ ]:
def calculate_error_rates(ref_file, hyp_file):
    with open(ref_file) as f1, open(hyp_file) as f2:
        refs = [line.strip().split(' ', 1)[1] for line in f1]
        hyps = [line.strip().split(' ', 1)[1] for line in f2]

        total_wer = 0
        total_cer = 0

        for ref, hyp in zip(refs, hyps):
            #WER calculation
            ref_words = ref.split()
            hyp_words = hyp.split()
            word_edits = editdistance.eval(ref_words, hyp_words)
            wer = word_edits / len(ref_words) if ref_words else 1
            total_wer += wer

            #CER calculation
            char_edits = editdistance.eval(ref, hyp)
            cer = char_edits / len(ref) if ref else 1
            total_cer += cer

        avg_wer = (total_wer / len(refs)) * 100
        avg_cer = (total_cer / len(refs)) * 100

        return avg_wer, avg_cer

wer, cer = calculate_error_rates(
    'data/test/text',
    'exp/marathi_asr/decode_test/1best_recog/text'
)
print(f"WER: {wer:.2f}%")
print(f"CER: {cer:.2f}%")

WER: 100.00%
CER: 26.17%


#Testing
##Common Voice 11 Marathi

In [ ]:
from datasets import load_dataset

RESAMPLED_AUDIO_DIR = "resampled_audio"
os.makedirs(RESAMPLED_AUDIO_DIR, exist_ok=True)

def resample_audio(input_path, output_path, target_sr=16000):
    audio, sr = librosa.load(input_path, sr=None)
    if sr != target_sr:
        audio = librosa.resample(y=audio, orig_sr=sr, target_sr=target_sr)
    sf.write(output_path, audio, target_sr)
    return output_path

#loading Common Voice 11
cv_train = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="train")
cv_test = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test")
cv_valid = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="validation")

combined_data = {
    'path': [],
    'sentence': []
}

for dataset in [cv_train, cv_test, cv_valid]:
    for item in dataset:
        original_path = item['audio']['path']
        filename = Path(original_path).name
        resampled_path = os.path.join(RESAMPLED_AUDIO_DIR, filename)

        if not os.path.exists(resampled_path):
            resampled_path = resample_audio(original_path, resampled_path)

        combined_data['path'].append(resampled_path)
        combined_data['sentence'].append(item['sentence'])

#splitting data into training-testing-validation splits with seed = 64
train_paths, temp_paths, train_sentences, temp_sentences = train_test_split(
    combined_data['path'], combined_data['sentence'],
    test_size=0.2, random_state=64
)

valid_paths, test_paths, valid_sentences, test_sentences = train_test_split(
    temp_paths, temp_sentences,
    test_size=0.5, random_state=64
)

print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(valid_paths)}")
print(f"Test samples: {len(test_paths)}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2819it [00:00, 122786.68it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2237it [00:00, 118470.90it/s]


Train samples: 4594
Validation samples: 574
Test samples: 575


In [ ]:
os.makedirs("cv_data/cv_test", exist_ok=True)
with open("cv_data/cv_test/wav.scp", "w") as wav_file, \
     open("cv_data/cv_test/text", "w") as text_file, \
     open("cv_data/cv_test/utt2spk", "w") as utt2spk_file:

    for idx, (path, text) in enumerate(zip(test_paths, test_sentences)):
        uttid = f"utterance_{idx}"
        wav_file.write(f"{uttid} {path}\n")
        text_file.write(f"{uttid} {text}\n")
        utt2spk_file.write(f"{uttid} global\n")



In [ ]:
!python -m espnet2.bin.asr_inference \
    --asr_model_file exp/marathi_asr/valid.loss.best.pth \
    --data_path_and_name_and_type cv_data/cv_test/wav.scp,speech,sound \
    --output_dir exp/marathi_asr/cv_decode_test \
    --allow_variable_data_keys true

Streaming output truncated to the last 5000 lines.
2024-11-30 15:45:32,960 (beam_search:483) INFO: best hypo: त्याहेळीझालेद्यागोध्लामधीदगांधीरुन्हासावंक्रीयोलागाहे

2024-11-30 15:45:32,965 (asr_inference:509) INFO: speech length: 66816
2024-11-30 15:45:33,009 (beam_search:428) INFO: decoder input length: 130
2024-11-30 15:45:33,009 (beam_search:429) INFO: max output length: 130
2024-11-30 15:45:33,009 (beam_search:430) INFO: min output length: 0
2024-11-30 15:45:33,011 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
2024-11-30 15:45:33,012 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
2024-11-30 15:45:35,962 (beam_search:444) INFO: end detected at 43
2024-11-30 15:45:35,963 (beam

In [ ]:
wer, cer = calculate_error_rates(
    'cv_data/cv_test/text',
    'exp/marathi_asr/cv_decode_test/1best_recog/text'
)

print(f"WER: {wer:.2f}%")
print(f"CER: {cer:.2f}%")

with open("cv_test_results.txt", "w") as f:
    f.write(f"WER: {wer:.2f}%\n")
    f.write(f"CER: {cer:.2f}%\n")

WER: 100.00%
CER: 63.26%
